In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import KFold
from d2l import torch as d2l


In [2]:
# 数据预处理
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)
 


In [3]:
# 定义简单神经网络模型
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28*28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 10),
        )
 
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
 


In [4]:
# K折交叉验证设置
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
# 训练和验证循环
def train_and_validate(model, train_loader, val_loader, num_epochs=10):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
     
    for epoch in range(num_epochs):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
     
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    return correct / total
 


In [6]:
fold_accuracies = []
for fold, (train_idx, val_idx) in enumerate(kf.split(mnist_train)):
    print(f"\nFold {fold + 1}")
     
    # 创建数据加载器
    train_subset = Subset(mnist_train, train_idx)
    val_subset = Subset(mnist_train, val_idx)
     
    train_loader = DataLoader(train_subset, batch_size=256, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_subset, batch_size=256, shuffle=False, num_workers=4)
     
    # 初始化模型
    model = SimpleModel().to(device)
     
    # 训练并验证
    accuracy = train_and_validate(model, train_loader, val_loader)
    fold_accuracies.append(accuracy)
    print(f"Fold {fold+1} Validation Accuracy: {accuracy*100:.2f}%")
 
# 输出结果
print("\nK-Fold Cross Validation Results:")
for fold, acc in enumerate(fold_accuracies):
    print(f"Fold {fold+1}: {acc*100:.2f}%")
print(f"Average Accuracy: {sum(fold_accuracies)/k*100:.2f}%")



Fold 1
Fold 1 Validation Accuracy: 88.60%

Fold 2
Fold 2 Validation Accuracy: 89.33%

Fold 3
Fold 3 Validation Accuracy: 89.79%

Fold 4
Fold 4 Validation Accuracy: 89.38%

Fold 5
Fold 5 Validation Accuracy: 89.36%

K-Fold Cross Validation Results:
Fold 1: 88.60%
Fold 2: 89.33%
Fold 3: 89.79%
Fold 4: 89.38%
Fold 5: 89.36%
Average Accuracy: 89.29%
